# 06. PyTorch Transfer Learning
Taking the parameters of what one model has learned on another dataset and applying it on our own problem...

Pretrained models are also called as foundation models.

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)  # want 0.13+

2.2.1+cu121
0.17.1+cu121


In [2]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Total 4056 (delta 0), reused 0 (delta 0), pack-reused 4056
Receiving objects: 100% (4056/4056), 646.90 MiB | 23.50 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Updating files: 100% (248/248), done.


In [4]:
# Setup device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 1. Get Data
We need our pizza, sushi and steak data to build a transfer model on.

In [6]:
import os
import zipfile

from pathlib import Path

import requests

# Setup data path and image path
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'  # Images from a subset of classes from the Food101 dataset.

# If the image folder doesn't exist, download it and create folder...
if image_path.is_dir():
  print(f'{image_path} directory exists, skipping re-download...')
else:
  print(f'Did not find {image_path}, downloading it...')
  image_path.mkdir(parents=True, exist_ok=True)

  # Downloading the data
  with open(data_path/'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    print('Downloading pizza, steak, sushi data...')
    f.write(request.content)

  # Unzip the data
  with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
    print('Unzipping pizza, steak and sushi data')
    zip_ref.extract_all(image_path)

  # Deleting the zip file
  os.remove(data_path/'pizza_steak_sushi.zip')

data/pizza_steak_sushi directory exists, skipping re-download...


In [7]:
train_dir = image_path / 'train'
test_dir = image_path / 'test'
train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))